In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import zscore
#from scipy.stats.mstats import winsorize
import numpy as np
#import itertools
import warnings
warnings.filterwarnings("ignore")

In [1]:
#Functions :
#convert lists values to new columns with this no need to change column type to string or NAN to [] values normally requires before applying below.
def list_to_columns(df,col): 
    dftemp = pd.DataFrame([pd.Series(x) for x in df[col]])
    column_names= []
    max_element = len(max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len))
    for i in range(max_element):
        column_names.append(col + str(i+1))
    dftemp.columns = column_names
    df = df.drop(col,axis=1)
    if max_element == 1:
        dftemp.rename(columns={col+'1': col},inplace = True)
    df_added = pd.concat([df, dftemp], axis=1, sort=False)
    return df_added

#Convert list of features as new columns and check if that feature exist on that car or not.
def list_to_feat_cols(df,col_list):
    dftemp = pd.DataFrame()
    for col in col_list:
        a = df[col].explode().unique()
        max_list = sorted([i for i in a if not pd.isnull(list(a)[list(a).index(i)])])
        for i in range(len(max_list)):
            dftemp[f"{col} {str(i+1)} {max_list[i]}"] = df[col].apply(lambda x : True if (type(x) == list and max_list[i] in x) \
               else (False if (type(x) == list and max_list[i] not in x) else x ))
        df = df.drop(col,axis=1)
    df_added = pd.concat([df, dftemp], axis=1, sort=False)
    return df_added


# check a column if it contains list of values. returns 2 list that we need to define
def check_is_list(df):
    list_of_columns = []
    non_list_of_columns = []
    for i in df.columns:
        counter_list=0
        counter_non_list=0
        for j in df[i].values:
            if isinstance(j, list):
                counter_list+=1
            else:
                counter_non_list+=1
        if counter_list > 0:
            list_of_columns.append(i)
        else :
            non_list_of_columns.append(i)
    return list_of_columns,non_list_of_columns
        
#get the df colon without outliers specified
def get_df_wo_outlier(df,col,range=1.5):
    Q1 = df[col].quantile(q=0.25)
    Q3 = df[col].quantile(q=0.75)
    IQR = Q3-Q1
    return df[col][~((df[col] <  Q1-(range*IQR)) |(df[col] > Q3+(range*IQR))) ]

#to have a look at data, which columns contain what and how many.
def get_all_df(df):
    for i in df.columns:
        print(df[i].value_counts(dropna = False).rename_axis(i +' unique_values').reset_index(name=i +' counts'))

#quick look on a column        
def get_col_details(df,col):
    print("VALUE COUNTS")
    print(df[col].value_counts(dropna=False))
    print("-----------------------\n","UNIQUE VALUES")
    print(df[col].unique())
    print("-----------------------\n","NULL SUM")
    print(df[col].isnull().sum())
    print("-----------------------\n","NULL Percentage")
    print(df[col].isnull().sum()*100 /df[col].index.stop )
    print("-----------------------\n","COLUMN with NONE-NULL-NAN")
    print(df.query(f'{col} != {col}')[col])
    
#updates values NAN to empty list : []
def Nan_to_list(df,col):
    Nan_rows = df[col].isnull()
    df.loc[Nan_rows,col] = df.loc[Nan_rows,col].apply(lambda x : [])
    
def outlier_zscore(df, col, min_z=1, max_z = 5, step = 0.1, print_list = False):
    z_scores = zscore(df[col].dropna())
    threshold_list = []
    for threshold in np.arange(min_z, max_z, step):
        threshold_list.append((threshold, len(np.where(z_scores > threshold)[0])))
        df_outlier = pd.DataFrame(threshold_list, columns = ['threshold', 'outlier_count'])
        df_outlier['pct'] = (df_outlier.outlier_count - df_outlier.outlier_count.shift(-1))/df_outlier.outlier_count*100
    plt.plot(df_outlier.threshold, df_outlier.outlier_count)
    best_treshold = round(df_outlier.iloc[df_outlier.pct.argmax(), 0],2)
    outlier_limit = int(df[col].dropna().mean() + (df[col].dropna().std()) * df_outlier.iloc[df_outlier.pct.argmax(), 0])
    percentile_threshold = stats.percentileofscore(df[col].dropna(), outlier_limit)
    plt.vlines(best_treshold, 0, df_outlier.outlier_count.max(), 
               colors="r", ls = ":"
              )
    plt.annotate("Zscore : {}\nValue : {}\nPercentile : {}".format(best_treshold, outlier_limit, 
                                                                   (np.round(percentile_threshold, 3), 
                                                                    np.round(100-percentile_threshold, 3))), 
                 (best_treshold, df_outlier.outlier_count.max()/2))
    #plt.show()
    if print_list:
        print(df_outlier)
    return (plt, df_outlier, best_treshold, outlier_limit, percentile)

def outlier_inspect(df, col, min_z=1, max_z = 5, step = 0.5, max_hist = None, bins = 50):
    fig = plt.figure(figsize=(20, 6))
    fig.suptitle(col, fontsize=16)
    plt.subplot(1,3,1)
    if max_hist == None:
        sns.distplot(df[col], kde=False, bins = 50)
    else :
        sns.distplot(df[df[col]<=max_hist][col], kde=False, bins = 50)
    plt.subplot(1,3,2)
    sns.boxplot(df[col])
    plt.subplot(1,3,3)
    z_score_inspect = outlier_zscore(df, col, min_z=min_z, max_z = max_z, step = step)
    plt.show()

In [3]:
# Fills missing values with combination of columns, Follows priority 
#firstly tries 3 column combination and fills nulls and applies remaining nulls with 2 colum combination and so on.
def fill_most_with1_temp(df,target_col,ref_col1):
    df_temp = df[[ref_col1,target_col]]
    for group1 in list(df_temp[ref_col1].unique()):
        grp_inx = list(df_temp[(df_temp[ref_col1]==group1)][target_col].index)
        try:
            fill_value = df_temp[(df_temp[ref_col1] == group1)][target_col].mode()[0]
        except:
            fill_value = 'NaN'
            continue
        df_temp[target_col].iloc[grp_inx] = df_temp[target_col].iloc[grp_inx].fillna(fill_value)
    return df_temp
def fill_most_with1_perm(df,target_col,ref_col1):
    for group1 in list(df[ref_col1].unique()):
        grp_inx = list(df[(df[ref_col1]==group1)][target_col].index)
        try:
            fill_value = df[(df[ref_col1] == group1)][target_col].mode()[0]
        except:
            fill_value = 'NaN'
            continue
        df[target_col].iloc[grp_inx] = df[target_col].iloc[grp_inx].fillna(fill_value)
    return df
def fill_most_with2_temp(df,target_col,col1, col2):
    df_temp = df[[col1,col2,target_col]]
    for group1 in list(df_temp[col1].unique()):
        for group2 in list(df_temp[df_temp[col1]==group1][col2].unique()):
            grp_inx = list(df_temp[(df_temp[col1]==group1) & (df_temp[col2] == group2)][target_col].index)
            try:
                fill_value = df_temp[(df_temp[col1] == group1) & (df_temp[col2] == group2)][target_col].mode()[0]
            except:
                fill_value = 'NaN'
                continue
            df_temp[target_col].iloc[grp_inx] = df_temp[target_col].iloc[grp_inx].fillna(fill_value)  
    return df_temp
def fill_most_with2_perm(df,target_col, col1, col2):
    for group1 in list(df[col1].unique()):
        for group2 in list(df[df[col1]==group1][col2].unique()):
            grp_inx = list(df[(df[col1]==group1) & (df[col2] == group2)][target_col].index)
            try:
                fill_value = df[(df[col1] == group1) & (df[col2] == group2)][target_col].mode()[0]
            except:
                fill_value = 'NaN'
                continue
            df[target_col].iloc[grp_inx] = df[target_col].iloc[grp_inx].fillna(fill_value)  
    return df
#df[(df["Make"] == "Audi") & (df["Model2"] == "A3")  & (df["Displacement (cc)"] == 1.5)]["body_type"].mode()[0]
def fill_most_with3_temp(df,target_col,col1, col2, col3):
    df_temp = df[[col1,col2,col3,target_col]]
    for group1 in list(df_temp[col1].unique()):
        for group2 in list(df_temp[df_temp[col1]==group1][col2].unique()):
            for group3 in list(df_temp[(df_temp[col1]==group1) & (df_temp[col2]==group2)][col3].unique()):
                grp_inx = list(df_temp[(df_temp[col1]==group1) & (df_temp[col2] == group2)& (df_temp[col3]==group3)][target_col].index)
                try:
                    fill_value = df_temp[(df_temp[col1] == group1) & (df_temp[col2] == group2) & (df_temp[col3] == group3)][target_col].mode()[0]
                except:
                    fill_value = 'NaN'
                    continue
                df_temp[target_col].iloc[grp_inx] = df_temp[target_col].iloc[grp_inx].fillna(fill_value)  
    return df_temp



def fill_most_with3_perm(df,target_col,col1, col2, col3):
    for group1 in list(df[col1].unique()):
        for group2 in list(df[df[col1]==group1][col2].unique()):
            for group3 in list(df[(df[col1]==group1) & (df[col2] == group2)][col3].unique()):
                grp_inx = list(df[(df[col1]==group1) & (df[col2] == group2)& (df[col3]==group3)][target_col].index)
                try:
                    fill_value = df[(df[col1] == group1) & (df[col2] == group2) & (df[col3] == group3)][target_col].mode()[0]
                except:
                    fill_value = 'NaN'
                    continue
                df[target_col].iloc[grp_inx] = df[target_col].iloc[grp_inx].fillna(fill_value)  
    return df

def fill_priority_perm(df,target_col,col1, col2, col3):
    fill_most_with3_perm(df,target_col,col1, col2, col3)
    fill_most_with2_perm(df,target_col,col1, col2)
    fill_most_with1_perm(df,target_col,col1)
    return df

def fill_priority_temp(df,target_col,col1, col2, col3):
    df_temp1 = fill_most_with3_temp(df,target_col,col1, col2, col3)
    df_temp2 = fill_most_with2_temp(df_temp1,target_col,col1, col2)
    df_temp3 = fill_most_with1_temp(df_temp2,target_col,col1)
    return df_temp3
   

In [13]:
df = pd.read_csv("checkpoint1.csv",index_col =False)

In [14]:
df

,make_model,body_type,price,vat,km,registration,hp (kW),Comfort & Convenience 1 Air conditioning,Comfort & Convenience 2 Air suspension,Comfort & Convenience 3 Armrest,...,Cylinders,Drive chain,Fuel2,CO2 Emission (g CO2/km (comb)),Emission Class1,Gears,Country version,Consumption l/100 km (comb),Consumption l/100 km (city),Consumption l/100 km (country)
0,Audi A1,Sedans,15770,VAT deductible,56013,01/2016,66,True,False,True,...,3,front,Diesel (Particulate Filter),99,6,5,Germany,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000,03/2017,141,True,False,False,...,4,front,Gasoline,129,6,7,Germany,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450,02/2016,85,True,False,False,...,3,front,Diesel (Particulate Filter),99,6,5,Germany,3.8,4.4,3.4
3,Audi A1,Sedans,14500,VAT deductible,73000,08/2016,66,False,True,True,...,3,front,Diesel (Particulate Filter),99,6,6,Germany,3.8,4.3,3.5
4,Audi A1,Sedans,16790,VAT deductible,16200,05/2016,66,True,False,True,...,3,front,Diesel (Particulate Filter),109,6,5,Germany,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,0,01/2019,147,True,False,False,...,4,front,Diesel (Particulate Filter),139,6,6,Germany,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900,01/2019,165,True,False,False,...,4,front,Super 95 / Super Plus 98 (Particulate Filter),168,6,7,Germany,7.4,7.4,5.3
15916,Renault Espace,Van,39875,VAT deductible,15,03/2019,146,True,False,True,...,4,front,Diesel,139,6,6,Austria,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10,06/2019,147,True,False,False,...,4,front,Diesel,139,6,6,Germany,5.3,6.2,4.7


In [16]:
for i in df.columns:
    print(df[i].isnull().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [19]:
df.km.value_counts(dropna=False)

10        1045
0         1043
1          367
5          170
50         148
          ... 
20615        1
141448       1
8329         1
84108        1
16364        1
Name: km, Length: 6689, dtype: int64

In [25]:
df.groupby("make_model")["km"].mean()

make_model
Audi A1           21728.625478
Audi A2           26166.000000
Audi A3           38295.754601
Opel Astra        36613.501979
Opel Corsa        24785.789094
Opel Insignia     38146.994226
Renault Clio      28948.030451
Renault Duster       47.529412
Renault Espace    33413.927346
Name: km, dtype: float64

In [29]:
df.groupby("make_model")["km"].transform("mean")

0        21728.625478
1        21728.625478
2        21728.625478
3        21728.625478
4        21728.625478
             ...     
15914    33413.927346
15915    33413.927346
15916    33413.927346
15917    33413.927346
15918    33413.927346
Name: km, Length: 15919, dtype: float64

In [30]:
df

,make_model,body_type,price,vat,km,registration,hp (kW),Comfort & Convenience 1 Air conditioning,Comfort & Convenience 2 Air suspension,Comfort & Convenience 3 Armrest,...,Cylinders,Drive chain,Fuel2,CO2 Emission (g CO2/km (comb)),Emission Class1,Gears,Country version,Consumption l/100 km (comb),Consumption l/100 km (city),Consumption l/100 km (country)
0,Audi A1,Sedans,15770,VAT deductible,56013,01/2016,66,True,False,True,...,3,front,Diesel (Particulate Filter),99,6,5,Germany,3.8,4.3,3.5
1,Audi A1,Sedans,14500,Price negotiable,80000,03/2017,141,True,False,False,...,4,front,Gasoline,129,6,7,Germany,5.6,7.1,4.7
2,Audi A1,Sedans,14640,VAT deductible,83450,02/2016,85,True,False,False,...,3,front,Diesel (Particulate Filter),99,6,5,Germany,3.8,4.4,3.4
3,Audi A1,Sedans,14500,VAT deductible,73000,08/2016,66,False,True,True,...,3,front,Diesel (Particulate Filter),99,6,6,Germany,3.8,4.3,3.5
4,Audi A1,Sedans,16790,VAT deductible,16200,05/2016,66,True,False,True,...,3,front,Diesel (Particulate Filter),109,6,5,Germany,4.1,4.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,0,01/2019,147,True,False,False,...,4,front,Diesel (Particulate Filter),139,6,6,Germany,5.3,6.2,4.7
15915,Renault Espace,Van,39885,VAT deductible,9900,01/2019,165,True,False,False,...,4,front,Super 95 / Super Plus 98 (Particulate Filter),168,6,7,Germany,7.4,7.4,5.3
15916,Renault Espace,Van,39875,VAT deductible,15,03/2019,146,True,False,True,...,4,front,Diesel,139,6,6,Austria,5.3,6.2,4.7
15917,Renault Espace,Van,39700,VAT deductible,10,06/2019,147,True,False,False,...,4,front,Diesel,139,6,6,Germany,5.3,6.2,4.7
